In [1]:
import copy

class Node:
   

    def __init__(self, position, moves, heuristic):
       
        self._position = position
        self._moves = moves
        self._heuristic = heuristic
        self._hScore = None

    def getPosition(self):
       
        return copy.deepcopy(self._position)

    def getGScore(self):
        
        return len(self._moves)

    def getHScore(self):
        
        if self._hScore is None:
            self._hScore = self._heuristic.compute(self)
        return self._hScore

    def getFScore(self):
        
        return self.getGScore() + self.getHScore()

    def getMoves(self):
        
        return copy.copy(self._moves)

    def getHeuristic(self):
        
        return self._heuristic

    def getCoordByValue(self, value):
        
        i = 0
        for row in self._position:
            j = 0
            for cell in row:
                if cell == value:
                    return [i, j]
                j += 1
            i += 1
            


class NodeBuilder:
    

    def getChildNodes(self, node):
        
        children = []
        iSpace,jSpace = node.getCoordByValue(0)
        for i in range(-1, 2):
            for j in range(-1, 2):
                if i * j != 0 or i == j:
                    continue
                iSwap, jSwap = iSpace + i, jSpace + j
                if not (0 <= iSwap <= 3) or not (0 <= jSwap <= 3):
                    continue
                position = node.getPosition()
                position[iSpace][jSpace] = position[iSwap][jSwap]
                position[iSwap][jSwap] = 0
                moves = node.getMoves()
                moves.append(self._getMoveNameFromDelta(i, j))
                child = Node(
                    position,
                    moves,
                    node.getHeuristic()
                )
                children.append(child)
        return children

    def _getMoveNameFromDelta(self, iDelta, jDelta):
        if iDelta == -1:
            return 'up'
        if iDelta == 1:
            return 'down'
        if jDelta == -1:
            return 'left'
        if jDelta == 1:
            return 'right'

class ManhattanDistance:
    

    def __init__(self):
        self._goal = Node([
            [ 1,  2,  3,  4],
            [ 5,  6,  7,  8],
            [ 9, 10, 11, 12],
            [13, 14, 15,  0]
        ], [], self)

    def compute(self, node):
        
        score = 0
        for value in range(1, 16):
            iGoal, jGoal = self._goal.getCoordByValue(value)
            iActual, jActual = node.getCoordByValue(value)
            score += abs(iGoal - iActual) + abs(jGoal - jActual)
        return score

    

    

class NodePool:
    

    def __init__(self):
        self._pool = []
        self._history = {}

    def add(self, node):
        
        if str(node.getPosition()) in self._history:
            # duplicate entry
            return
        self._history[str(node.getPosition())] = True
        self._insort(node)

    def pop(self):
        
        return self._pool.pop(0)

    def isEmpty(self):
        
        return len(self._pool) == 0

    def _insort(self, node):
        
        lo = 0
        hi = len(self._pool)
        while lo < hi:
            mid = (lo+hi)//2
            if node.getFScore() < self._pool[mid].getFScore(): hi = mid
            else: lo = mid + 1
        self._pool.insert(lo, node)
        
        
class AStar:
    

    def __init__(self, heuristic):
        self._nodePool = NodePool()
        self._nodeBuilder = NodeBuilder()
        self._heuristic = heuristic

    def solve(self, position):
        
        self._bootstrap(position)
        while not self._nodePool.isEmpty():
            # Pop best node from priority queue
            currentNode = self._nodePool.pop()
            if currentNode.getHScore() == 0:
                # Solution found!
                return currentNode.getMoves()
            # Compute child nodes and add them to the queue
            children = self._nodeBuilder.getChildNodes(currentNode)
            for child in children:
                self._nodePool.add(child)
        # No solution has been found
        return None

    def _bootstrap(self, position):
       
        node = Node(position, [], self._heuristic)
        self._nodePool.add(node)
        
        

        
import time
from pprint import pprint

heuristic = ManhattanDistance()
astar = AStar(heuristic)
start = [
            [ 1,  6,  7,  5],
            [ 9,  3, 10,  2],
            [13,  8,  4, 12],
            [14, 11, 15,  0]
        ]

startTime = time.time()
startComplexity = heuristic.compute(
    Node(start, [], None)
)

result = astar.solve(start)

if result is None:
    print('No solution found')
else:
    pprint(result)
    print('Heuristic said at least %d moves were needed.' % startComplexity)
    print('Actually solution is %d moves away. Best solution found guaranteed!' % len(result))
    print('Solved in %d seconds.' % (time.time() - startTime))

['up',
 'left',
 'left',
 'up',
 'right',
 'up',
 'right',
 'down',
 'down',
 'left',
 'up',
 'up',
 'right',
 'down',
 'down',
 'left',
 'left',
 'up',
 'right',
 'down',
 'right',
 'down',
 'left',
 'left',
 'left',
 'up',
 'up',
 'right',
 'up',
 'right',
 'down',
 'down',
 'down',
 'right']
Heuristic said at least 24 moves were needed.
Actually solution is 34 moves away. Best solution found guaranteed!
Solved in 8 seconds.
